# Example 3. Sentiment training with IMBd data, out-of-core learning
---

### Import package and function definition

In [3]:
import re
import nltk
import pandas as pd
import numpy  as np

In [4]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text) # remove html flag, e.g. <br />
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower())+' '.join(emoticons).replace('-','')
    tokenized = [w for w in text.split() if w not in stop]
    return text

[nltk_data] Downloading package stopwords to /Users/Alpha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label
# test:
next(stream_docs(path='../data/imbd.csv.train'))

('"... but the trouble of this production is that it\'s very far from a good musical.<br /><br />Granted, one can\'t always expect the witty masters like Sondheim or Bernstein or Porter; yet the music of this piece makes even Andrew Lloyd Webber look witty. It\'s deadly dull and uninventive (with one or two exceptions) and just after I watched it I couldn\'t recall a single significant melody - which is rather tragic coming from someone who learned the whole Another Hundred People from three listenings.<br /><br />It is also strangely un-theatrical. It takes place on an incredibly large stage (one really has to feel sorry for those people in front rows who broke their necks in order to see something happening 50 meters on the right or 100 meters on the left) and does absolutely nothing with it. When there\'s supposed to be one person singing on-stage, that\'s just what you get - and the rest of the enormeous stage is empty. For me as an aspiring theatre director it was almost painful t

In [7]:
def get_minibatch(docs_stream, size):
    docs, y = [], []
    try: 
        for _ in range(size):
            text, label = next(docs_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

### Train

In [8]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [ ]:
vect = HashingVectorizer(decode_error='ignore',
                         n_feature=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
docs_stream = stream_docs(path='../data/imbd.csv.')